# Evaluate ROV status of visible space

This workbook evaluates the ROV status for all riswhois entries.


In [1]:
# Add parent dir to path
import os,sys

sys.path.insert(0,os.environ['PWD'])

In [2]:
import bz2
import json
import ipaddress
import netaddr

import altair as alt
import pandas as pd
import requests

from rpki_analysis.riswhois import RisWhoisLookup, read_ris_dump
from rpki_analysis.rov import RouteOriginAuthorizationLookup, rov_validity, rov_validity_verbose

In [3]:
# https://www.ris.ripe.net/dumps/riswhoisdump.IPv4.gz
# https://www.ris.ripe.net/dumps/riswhoisdump.IPv6.gz
ris_v4_gz = requests.get("https://www.ris.ripe.net/dumps/riswhoisdump.IPv4.gz").content
ris_v6_gz = requests.get("https://www.ris.ripe.net/dumps/riswhoisdump.IPv6.gz").content


In [4]:
# Atlas Probe archive
atlas_probes_bz2 = requests.get("https://ftp.ripe.net/ripe/atlas/probes/archive/meta-latest").content
atlas_probes = pd.DataFrame(json.loads(bz2.decompress(atlas_probes_bz2))['objects'])

In [5]:
ris_v4 = read_ris_dump("https://www.ris.ripe.net/dumps/riswhoisdump.IPv4.gz")
ris_v6 = read_ris_dump("https://www.ris.ripe.net/dumps/riswhoisdump.IPv6.gz")

RIS dump contains row(s) with AS_SET! These will never be RPKI valid (https://tools.ietf.org/html/rfc6907#section-7.1.8)
RIS dump contains row(s) with AS_SET! These will never be RPKI valid (https://tools.ietf.org/html/rfc6907#section-7.1.8)


In [6]:
realistic_announcements_v4 = ris_v4[ris_v4.seen_by_peers > 50].copy()
realistic_announcements_v4_set = netaddr.IPSet(realistic_announcements_v4.prefix)

In [7]:
def announced_in_prefix(*prefixes: str) -> netaddr.IPSet:
    mask = netaddr.IPSet(prefixes)
    return realistic_announcements_v4_set & mask

In [8]:
#announced_frac = pd.DataFrame([{"prefix": f"{x}.0.0.0/8", "announced_fraction": (realistic_announcements_v4_set & netaddr.IPSet([f"{x}.0.0.0/8"])).size/2**24} for x in range(256)])

In [10]:
#announced_frac

In [11]:
#announced_frac.plot.hist(bins=32)

In [12]:
# Get RPKI valids
validated_objects = requests.get("https://rpki-validator.ripe.net/json").json()
rpki_roas = pd.DataFrame(validated_objects['roas'])
# And add address family
rpki_roas['af'] = rpki_roas.prefix.map(lambda p: ipaddress.ip_network(p).version)
# And prefix length (to chart later)
rpki_roas['prefix_length'] = rpki_roas.prefix.map(lambda p: ipaddress.ip_network(p).prefixlen)

rpki_roas_v4 = rpki_roas[rpki_roas.af == 4]
rpki_roas_v6 = rpki_roas[rpki_roas.af == 6]

# And build patricia tries
roa_lookup_v4 = RouteOriginAuthorizationLookup(rpki_roas_v4)
roa_lookup_v6 = RouteOriginAuthorizationLookup(rpki_roas_v6)


In [17]:
# calculate ROV status
realistic_announcements_v4['rov'] = realistic_announcements_v4.apply(lambda x: rov_validity(x, roa_lookup_v4), axis=1)

In [19]:
def rov_status_for_prefix(df: pd.DataFrame) -> pd.DataFrame:
    """Calculate the 'best' ROV status for a prefix."""
    for status in ["valid", "unknown", "invalid"]:
        cur = df[df.rov == status]
        if len(cur) > 0:
            return cur.head(1)
    return pd.DataFrame()

In [18]:
realistic_announcements_v4

,origin,prefix,seen_by_peers,prefix_length,rov
64,13335,1.0.0.0/24,414.0,24,invalid
65,38803,1.0.4.0/22,416.0,22,invalid
66,38803,1.0.5.0/24,413.0,24,invalid
67,18144,1.0.64.0/18,376.0,18,unknown
68,23969,1.0.128.0/17,426.0,17,unknown
...,...,...,...,...,...
1051113,63199,223.255.250.0/24,366.0,24,unknown
1051114,63199,223.255.251.0/24,366.0,24,unknown
1051115,58519,223.255.252.0/24,364.0,24,invalid
1051116,58519,223.255.253.0/24,364.0,24,invalid


In [ ]:
announcements_rov_status = realistic_announcements_v4.groupby('prefix').apply(rov_status_for_prefix)

In [ ]:
announcements_rov_status

In [ ]:
# seen_by_peers_count_by_roa_validity = ris_v4.groupby(['roa_validity', 'seen_by_peers'], as_index=False).agg({'prefix': 'count'})

# rov_chart = alt.Chart(
#     seen_by_peers_count_by_roa_validity).mark_bar()\
# .encode(
#     alt.X("seen_by_peers:Q", bin=False),
#     alt.Color("roa_validity:N"),
#     y='prefix',
# ).properties(
#     title='Visibility versus ROV status',
#     width=300,
#     height=150,
# ).facet(
#     column='roa_validity',
# ).resolve_scale(
#     y='independent'
# )

# #rov_chart.save("ris_v4_rov_chart.png")
# display(rov_chart)